In [1]:
# pip install pymupdf4llm

In [2]:
import requests
import re
import json
import nltk
import fitz
import unicodedata
import pandas as pd

from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import spacy

# Baixar recursos necessários do NLTK
nltk.download("punkt")
nltk.download("rslp")

stemmer = RSLPStemmer()

stop_words = set(stopwords.words("portuguese"))

# Carrega o modelo de linguagem do spaCy para português
try:
    nlp = spacy.load("pt_core_news_sm")
except OSError:
    # Se o download falhar na célula de instalação, tenta novamente
    !python -m spacy download pt_core_news_sm
    nlp = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a-a-c\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\a-a-c\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
url = "https://dodf.df.gov.br/dodf/jornal/visualizar-pdf?pasta=2025|11_Novembro|DODF%20220%2019-11-2025|&arquivo=DODF%20220%2019-11-2025%20INTEGRA.pdf"

raw_name = url.split("arquivo=")[-1]
file_name = raw_name.replace("%20", " ")

response = requests.get(url)
response.raise_for_status()

with open(file_name, "wb") as f:
    f.write(response.content)

print("PDF salvo como:", file_name)


PDF salvo como: DODF 220 19-11-2025 INTEGRA.pdf


### Extração de Texto do PDF

In [4]:
doc = fitz.open('DODF 220 19-11-2025 INTEGRA.pdf')
texto_completo = []

for pagina in doc:
    # Extrai blocos de texto com coordenadas
    blocos = pagina.get_text("blocks")
    
    # Ordena por posição (top-left, depois top-right)
    blocos_ordenados = sorted(blocos, key=lambda b: (b[1], b[0]))
    
    texto_pagina = "\n".join([bloco[4] for bloco in blocos_ordenados if bloco[6] == 0]) 
    texto_completo.append(texto_pagina)

doc.close()
pdf_content = "".join(texto_completo)

### Extração de Texto

In [5]:
# ---- EXTRAÇÃO DAS SEÇÕES PRINCIPAIS (I, II, III) ----

def extract_main_sections(text):
    pattern = r"(SEÇÃO\s+[IVX]+)"
    partes = re.split(pattern, pdf_content)
    extract_sections = []
    for i in range(1, len(partes), 2):
        titulo = partes[i].strip()
        conteudo = partes[i+1].strip() if i+1 < len(partes) else ""
        # --- FILTROS IMPORTANTES ---
        
        # remover seções muito pequenas (sumário/capa)
        if len(conteudo) < 500:
            continue
        # ignorar sumário
        if conteudo[:50].upper().startswith("SUMÁRIO"):
            continue
        
        # evitar duplicações vazias ou irrelevantes
        if "PAG." in conteudo[:80]:
            continue
        # manter apenas SEÇÃO I, II e III
        if titulo not in ["SEÇÃO I", "SEÇÃO II", "SEÇÃO III"]:
            continue
        extract_sections.append({
            "secao": titulo,
            "conteudo": conteudo
        })

    return extract_sections

In [6]:
sections = extract_main_sections(pdf_content)
# print(json.dumps(sections, ensure_ascii=False, indent=2))

section_I = sections[0]["conteudo"]
section_II = sections[1]["conteudo"]
section_III = sections[2]["conteudo"]
print(len(section_III))

370725


### Seção 1

In [8]:
# REGEX DE TÍTULO DE ATO
TITLE_REGEX = r'^[A-ZÁÉÍÓÚÃÕÂÊÔÇ ]{2,}\s+N[º°oO\.]?\s*[\d\.\/-]+(?:,\s*DE\s+[^\n]+)?'
TITLE_PATTERN = re.compile(TITLE_REGEX)

# REGEX PARA META-INFO
META_REGEX = re.compile(
    r'^(?P<tipo>[A-ZÁÉÍÓÚÃÕÂÊÔÇ ]+)\s+N[º°oO\.]?\s*(?P<numero>[\d\.\/-]+)'
    r'(?:,\s*DE\s*(?P<data>[^\n]+))?',
    re.UNICODE
)

# EXTRAÇÃO DO ÓRGÃO EMISSOR
ORG_REGEX = re.compile(
    r'\b(GABINETE|SECRETARIA|MINISTÉRIO|SUPERINTENDÊNCIA|PREFEITURA|GOVERNO|CÂMARA|ASSEMBLEIA|TRIBUNAL|PROCURADORIA)[^\n]{0,80}',
    re.IGNORECASE
)

# FUNÇÃO PRINCIPAL
def extract_acts_with_metadata(section_text):
    lines = section_text.split("\n")

    acts = []
    current_title = None
    current_content = []

    for line in lines:
        stripped = line.strip()

        # Detectou título?
        if TITLE_PATTERN.match(stripped):
            if current_title:  
                acts.append(process_act(current_title, current_content))

            current_title = stripped
            current_content = []
            continue

        # Acumula conteúdo
        if current_title:
            current_content.append(line)

    # salva o último
    if current_title:
        acts.append(process_act(current_title, current_content))

    return acts


# PROCESSAR ATO INDIVIDUAL
def process_act(title, content_lines):
    meta = META_REGEX.match(title)

    tipo = meta.group("tipo").strip() if meta else None
    numero = meta.group("numero").strip() if meta else None
    data_raw = meta.group("data").strip() if meta and meta.group("data") else None

    # extrair ano da data
    ano = None
    if data_raw:
        ano_match = re.search(r'\b(19|20)\d{2}\b', data_raw)
        if ano_match:
            ano = int(ano_match.group(0))

    # detectar órgão emissor no conteúdo
    content_text = "\n".join(content_lines).strip()
    org_match = ORG_REGEX.search(content_text)
    orgao = org_match.group(0).strip() if org_match else None

    return {
        "tipo": tipo,
        "numero": numero,
        "data": data_raw,
        "ano": ano,
        "orgao": orgao,
        "titulo": title,
        "conteudo": content_text
    }


In [9]:
acts = extract_acts_with_metadata(section_I)
# print(json.dumps(acts, ensure_ascii=False, indent=2))

### Seção 2

In [13]:
text = section_II  # sua variável com todo o texto da seção II

verbs_list = [
    "NOMEAR","EXONERAR A PEDIDO","EXONERAR","DESIGNAR",
    "DISPENSAR","REMOVER","RECONDUZIR","RETIFICAR",
    "TORNAR SEM EFEITO","CONCEDER","PRORROGAR","CEDER"
]
verbs_regex = "|".join(sorted(verbs_list, key=lambda s: -len(s)))

# normalização leve
t = re.sub(r"\r\n?", "\n", text)
t = re.sub(r"\s+", " ", t)
t = re.sub(rf"(?i)\b({verbs_regex})\b", lambda m: "\n\n" + m.group(1).upper(), t)

# split em blocos por ato
blocks = [b.strip() for b in re.split(r"\n\s*\n", t) if b.strip()]

# -----------------------------
# REGEX CORRIGIDAS
# -----------------------------

# Nome em maiúsculas
name_re = re.compile(
    r"([A-ZÁÉÍÓÚÂÊÔÃÕÇ][A-ZÁÉÍÓÚÂÊÔÃÕÇ\.\-']+(?:\s+[A-ZÁÉÍÓÚÂÊÔÃÕÇ\.\-']+){1,6})"
)

# Matrícula
mat_re = re.compile(
    r"matr[ií]cula[:\s]*([\d\.\-Xx]+)", flags=re.IGNORECASE
)

# SIGRH
sigrh_re = re.compile(
    r"SIGRH[:\s]*([0-9]{4,12})", flags=re.IGNORECASE
)

# Símbolo (corrigido)
sim_re = re.compile(
    r"(?:Símbolo|SIMBOLO|SÍMBOLO)[:\s]*([A-Z]{1,4}[-\s]?\d{1,3})",
    flags=re.IGNORECASE
)

# Cargo genérico
cargo_re_1 = re.compile(
    r"Cargo(?: Público)?(?: em)?(?: de)?(?: Natureza Especial|[^,;.\n]{1,80})",
    flags=re.IGNORECASE
)

# Função (corrigido — fechamento de parêntesis validado)
role_re = re.compile(
    r"\b(?:de\s+)?([A-ZÁÉÍÓÚÂÊÔÃÕÇ][a-záéíóúâêôãõç]+(?:\s+[A-ZÁÉÍÓÚÂÊÔÃÕÇa-záéíóúâêôãõç]+){0,5})",
    flags=re.IGNORECASE
)

# Órgãos
org_keywords = [
    "Secretaria","Subsecretaria","Diretoria","Gerência","Gerencia",
    "Coordenação","Coordenação","Gabinete","Unidade","Conselho",
    "Administração Regional"
]

org_re = re.compile(
    r"(?:(?:da|do|das|dos)\s+)?("
    r"(?:" + r"|".join(org_keywords) + r")[^\.\,]{0,120})",
    flags=re.IGNORECASE
)

# -----------------------------
# FUNÇÃO PARA NORMALIZAR NOME
# -----------------------------

def title_case_name(s):
    s = s.strip()
    s = unicodedata.normalize('NFKD', s)
    parts = s.split()
    small = {"da","de","do","das","dos","e","da'", "do'"}
    out = []
    for i,p in enumerate(parts):
        pl = p.lower()
        if pl in small and i != 0:
            out.append(pl)
        else:
            out.append(p.capitalize())
    return " ".join(out).replace("  ", " ").strip()

# -----------------------------
# PROCESSAMENTO
# -----------------------------

entries = []

for blk in blocks:
    blk_orig = blk

    # detectar ato
    m_ato = re.match(rf"^\s*({verbs_regex})\b", blk, flags=re.IGNORECASE)
    if not m_ato:
        continue

    ato = m_ato.group(1).upper()
    content = blk[len(m_ato.group(0)):].strip()

    subblocks = re.split(
        r"(?<=\.)\s+|(?<=;)\s+|(?=(?:\bNOMEAR\b|\bEXONERAR\b|\bDESIGNAR\b|\bEXONERAR A PEDIDO\b))",
        content,
        flags=re.IGNORECASE
    )
    if not subblocks:
        subblocks = [content]

    for sb in subblocks:
        s = sb.strip()
        if not s:
            continue

        s_clean = re.sub(r"(?i)\bpor\s+estar\s+sendo\b[^\.,]*[\,]?", "", s)
        s_clean = s_clean.strip().lstrip(",").strip()

        names = name_re.findall(s_clean)

        if not names:
            m_after = re.match(
                r"^\s*([A-ZÁÉÍÓÚÂÊÔÃÕÇ][A-ZÁÉÍÓÚÂÊÔÃÕÇ\.\-']+(?:\s+[A-ZÁÉÍÓÚÂÊÔÃÕÇ\.\-']+){1,6})",
                s_clean
            )
            if m_after:
                names = [m_after.group(1)]

        if not names:
            continue

        # -----------------------------
        # Loop de múltiplos nomes
        # -----------------------------
        for nm in names:

            pos = s_clean.find(nm)
            if pos == -1:
                continue

            remaining = s_clean[pos + len(nm):].strip()

            matricula = None
            sigrh = None
            simbolos = []
            cargo = None
            orgao = None

            mm = mat_re.search(remaining)
            if mm:
                matricula = mm.group(1).strip()

            ms = sigrh_re.findall(remaining)
            if ms:
                sigrh = ms[0] if ato.startswith("EXONERAR") else ms[-1]

            sims = sim_re.findall(remaining)
            if sims:
                simbolos = [re.sub(r"[\s]+","-",x).upper() for x in sims]

            # -----------------------------
            # CARGO — TODAS AS REGEX CORRIGIDAS
            # -----------------------------

            mc = re.search(
                r"para\s+exercer\s+o\s+"
                r"(?:Cargo(?: Público)?(?: em)?(?: de)?(?: Natureza Especial|[^,;\.]+)"
                r"|Cargo em Comissão|Cargo Público em Comissão)"
                r"[\s,]*(?:,|de)?\s*([^,;\.]+)",
                s_clean,
                flags=re.IGNORECASE
            )

            if mc:
                cargo = mc.group(1).strip().rstrip(".,;")
            else:
                mc2 = re.search(
                    r"do\s+(Cargo(?: Público)?(?: em)?(?: de)?(?: Natureza Especial|[^,;\.]+)"
                    r"|Cargo em Comissão|Cargo Público em Comissão)"
                    r"[\s,]*(?:,|de)?\s*([^,;\.]+)",
                    s_clean,
                    flags=re.IGNORECASE
                )
                if mc2:
                    cargo = mc2.group(1).strip()
                else:
                    # REGEX corrigida — sem parênteses extra
                    mr = re.search(
                        r"\b(?:de|,)\s*([A-ZÁÉÍÓÚÂÊÔÃÕÇ][A-Za-záéíóúâêôãõçº°]+"
                        r"(?:\s+[A-ZÁÉÍÓÚÂÊÔÃÕÇa-záéíóúâêôãõçº°]+){0,5})",
                        remaining
                    )
                    if mr:
                        cand_role = mr.group(1).strip()
                        if len(cand_role.split()) <= 5:
                            cargo = cand_role

            # -----------------------------
            # ÓRGÃO
            # -----------------------------
            org_matches = org_re.findall(s_clean)
            if org_matches:
                last_org = org_matches[-1]
                if isinstance(last_org, tuple):
                    last_org = next((x for x in last_org if x), last_org[0])
                orgao = re.sub(r"^(da|do|das|dos)\s+", "", last_org.strip(), flags=re.IGNORECASE)

            # fallback para cargo após SIGRH/símbolo
            if not cargo:
                m_after_sym = re.search(
                    r"(?:Símbolo[:\s]*[A-Z0-9\-\s]+[,;\s]*)?"
                    r"(?:SIGRH[:\s]*\d+[,;\s]*)?(.*)$",
                    remaining
                )
                if m_after_sym:
                    maybe = m_after_sym.group(1).strip()
                    maybe_piece = maybe.split(",")[0].strip()
                    if len(maybe_piece.split()) <= 6 and len(maybe_piece) > 3:
                        if re.search(r"[A-Za-zÀ-ÿ]", maybe_piece):
                            cargo = maybe_piece

            clean_name = nm.strip()
            if re.match(rf"(?i)^(?:{verbs_regex})\b", clean_name):
                clean_name = re.sub(rf"(?i)^(?:{verbs_regex})\b\.?,?\s*", "", clean_name).strip()
            clean_name = title_case_name(clean_name)

            simbolo_main = None
            if simbolos:
                simbolos_norm = [re.sub(r"[^\w\-]","",s).upper() for s in simbolos]
                simbolo_main = simbolos_norm[-1] if ato.startswith("NOMEAR") else simbolos_norm[0]

            entry = {
                "uid": None,
                "secao": "SEÇÃO II",
                "ato": ato,
                "nome": clean_name,
                "matricula": matricula,
                "sigrh": sigrh,
                "simbolo": simbolo_main,
                "cargo": (cargo.strip() if cargo else None),
                "orgao": (orgao.strip() if orgao else None),
                "raw": s_clean
            }

            uid_base = f"{ato}-{matricula or clean_name.replace(' ','_')}"
            entry["uid"] = re.sub(r"[^\w\-_]","", uid_base)[:80]

            entries.append(entry)

# deduplicação
seen = set()
unique = []
for e in entries:
    key = (e.get("ato"), e.get("nome"), e.get("matricula"))
    if key in seen:
        continue
    seen.add(key)
    unique.append(e)

entries = unique


In [ ]:
print(json.dumps(entries, ensure_ascii=False, indent=2))